<a href="https://colab.research.google.com/github/Offcute/CF-DashApp/blob/main/Test_Rule_Based_Classification_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Rule Based Classification Problem

In [33]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/ISproject

# List files to make sure we're in the expected directory.
# Your output will look different, showing your own Drive files here.
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ISproject
 Back-up_dash_app.ipynb
 client_secrets.json
'Copy of Section2_DashApplication.ipynb'
 Copy_Section1_NewProductionSurvey.ipynb
 Copy_Section2_DashApplication.ipynb
 Copy_Section3_ProductPossibility.ipynb
 Copy_Test_FeatureSelection.ipynb
'Copy_Total responses categorized characteristic_database.csv'
 CorrelationExcel.xlsx
 customer-survey-feedback-0db2f1954d4e.json
 customer-survey-feedback-f58deed6af2f.json
 Merge_600138.csv
'Merge_600138 ROC curve .ipynb'
'Merge_600138ROC_curve .py'
 Merge_600138.SH.xlsx
'New Headphone Production Survey Complete.csv'
'New Headphone Production Survey for ML.csv'
 __pycache__
 RuleBased_Classification.py
 Section1_NewProductionSurvey.ipynb
 Section2_DashApplication.ipynb
 Test_FeatureSelection.ipynb
'Test Rule Based Classification Problem.ipynb'
 Test_Survey_ML.ipynb
'Total res

In [34]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)  # or 199
np.set_printoptions(threshold=np.inf)

df = pd.read_csv("New Headphone Production Survey Complete.csv")


In [35]:
df["Htype"] = df["HpAnswer"].copy()
df.shape

(72, 40)

In [36]:
agg_df = df.groupby(["Gender","Status","Age","Education","Occupation","Htype"])[["HpAnswer"]].count()
agg_df.shape

(48, 1)

In [37]:
agg_df = agg_df.reset_index()
agg_df = agg_df.sort_values(by=["Htype"])
agg_df.head(10)

,Gender,Status,Age,Education,Occupation,Htype,HpAnswer
24,Male,Married,28-35,Graduate,Employee,v1,1
19,Female,Single,28-35,Undergraduate,Employee,v1,4
43,Male,Single,28-35,Undergraduate,Employee,v1,1
40,Male,Single,28-35,Graduate,Employee,v1,1
6,Female,Married,36-45,Other education,Business Owner,v1,1
8,Female,Single,20-27,Graduate,Other occupation,v1,1
14,Female,Single,20-27,Undergraduate,Other occupation,v1,1
11,Female,Single,20-27,Undergraduate,Employee,v1,1
27,Male,Married,28-35,Undergraduate,Employee,v2,1
29,Male,Married,36-45,Graduate,Employee,v2,1


In [38]:
agg_df["customers_level_based"] = [f"{i[0]}_{i[1]}_{i[2]}_{i[3]}_{i[4]}" for i in agg_df.values]
#agg_df= agg_df.loc[:,["customers_level_based","HpAnswer"]].groupby("customers_level_based").agg({"HpAnswer": "sum"}).sort_values(by="HpAnswer", ascending=False).reset_index()
agg_df.head(10) # Just checking data again

,Gender,Status,Age,Education,Occupation,Htype,HpAnswer,customers_level_based
24,Male,Married,28-35,Graduate,Employee,v1,1,Male _Married _28-35 _Graduate _Employee
19,Female,Single,28-35,Undergraduate,Employee,v1,4,Female _Single _28-35 _Undergraduate _Employee
43,Male,Single,28-35,Undergraduate,Employee,v1,1,Male _Single _28-35 _Undergraduate _Employee
40,Male,Single,28-35,Graduate,Employee,v1,1,Male _Single _28-35 _Graduate _Employee
6,Female,Married,36-45,Other education,Business Owner,v1,1,Female _Married _36-45 _Other education_Business Owner
8,Female,Single,20-27,Graduate,Other occupation,v1,1,Female _Single _20-27 _Graduate _Other occupation
14,Female,Single,20-27,Undergraduate,Other occupation,v1,1,Female _Single _20-27 _Undergraduate _Other occupation
11,Female,Single,20-27,Undergraduate,Employee,v1,1,Female _Single _20-27 _Undergraduate _Employee
27,Male,Married,28-35,Undergraduate,Employee,v2,1,Male _Married _28-35 _Undergraduate _Employee
29,Male,Married,36-45,Graduate,Employee,v2,1,Male _Married _36-45 _Graduate _Employee


In [39]:
#agg_df[(agg_df.HpAnswer == agg_df.HpAnswer.max()) and (agg_df.Htype.equal("v1"))]
a = agg_df.loc[(agg_df["Htype"]=="v1")]
a = a[["customers_level_based","HpAnswer"]][(a.HpAnswer == a.HpAnswer.max())]
b = agg_df.loc[(agg_df["Htype"]=="v2")]
b = b[["customers_level_based","HpAnswer"]][(b.HpAnswer == b.HpAnswer.max())]
c = agg_df.loc[(agg_df["Htype"]=="v3")]
c = c[["customers_level_based","HpAnswer"]][(c.HpAnswer == c.HpAnswer.max())]

lst_max = a._append(b, ignore_index = True)
lst_max = lst_max._append(c, ignore_index = True)
lst_max


,customers_level_based,HpAnswer
0,Female _Single _28-35 _Undergraduate _Employee,4
1,Female _Single _28-35 _Graduate _Employee,5
2,Female _Single _20-27 _Undergraduate _Other occupation,4


In [40]:
b = agg_df.groupby(["customers_level_based"])[["Htype"]].nunique().reset_index()
b = b.loc[b['Htype']==1]

In [41]:
b["customers_level_based"].shape[0]


27

In [42]:
segment =agg_df[["customers_level_based","Htype"]].loc[agg_df['customers_level_based'].isin(b["customers_level_based"])]
segment = segment.sort_values(by=["Htype"])
segment = segment.reset_index(drop=True)
segment

,customers_level_based,Htype
0,Female _Married _36-45 _Other education_Business Owner,v1
1,Female _Single _20-27 _Graduate _Employee,v2
2,Female _Married _36-45 _High School _Other occupation,v2
3,Female _Married _36-45 _High School _Business Owner,v2
4,Female _Single _28-35 _Graduate _Business Owner,v2
5,Female _Single _28-35 _Graduate _Employee,v2
6,Female _Single _28-35 _Other education_Employee,v2
7,Female _Married _28-35 _Graduate _Other occupation,v2
8,Female _Single _36-45 _Undergraduate _Employee,v2
9,Female _Married _28-35 _Undergraduate _Employee,v2


In [43]:
segment.shape[0]

27

In [44]:
new_user = "Male _Single _36-45 _Undergraduate _Employee"
d =segment["Htype"].loc[segment['customers_level_based'].str.strip()==new_user]

print(d.values)


['v3']


In [45]:
c1 = segment.loc[segment['customers_level_based'].str.contains("Undergraduate _Employee")]
c1

,customers_level_based,Htype
8,Female _Single _36-45 _Undergraduate _Employee,v2
9,Female _Married _28-35 _Undergraduate _Employee,v2
13,Male _Married _36-45 _Undergraduate _Employee,v2
19,Male _Single _20-27 _Undergraduate _Employee,v3
26,Male _Single _36-45 _Undergraduate _Employee,v3


In [46]:
#c = round(c['Htype'].value_counts(normalize=True)*100,2)
p1 =  round(c1['Htype'].value_counts(normalize=True)*100,2).rename_axis('Htype').reset_index(name='Pros')
p1

,Htype,Pros
0,v2,60.0
1,v3,40.0


In [47]:
p1.Pros.max()

60.0

In [48]:
g = segment.loc[segment['customers_level_based'].str.contains("_36-45 _Other education_Business Owner")]
g

,customers_level_based,Htype
0,Female _Married _36-45 _Other education_Business Owner,v1


In [50]:
c2 = c1._append(g, ignore_index=True).sort_values(by='Htype')
c2

,customers_level_based,Htype
5,Female _Married _36-45 _Other education_Business Owner,v1
0,Female _Single _36-45 _Undergraduate _Employee,v2
1,Female _Married _28-35 _Undergraduate _Employee,v2
2,Male _Married _36-45 _Undergraduate _Employee,v2
3,Male _Single _20-27 _Undergraduate _Employee,v3
4,Male _Single _36-45 _Undergraduate _Employee,v3


In [51]:
p2 =  round(c2['Htype'].value_counts(normalize=True)*100,2).rename_axis('Htype').reset_index(name='Pros').sort_values(by='Htype')
p2

,Htype,Pros
2,v1,16.67
0,v2,50.00
1,v3,33.33
